In [ ]:
import json
import requests

import boto3

# 1. Fetch all the tech posts launching today on Product Hunt
Before fetching the daily Product Hunt posts, you need a developer token, an API key + API secret. Visit Product Hunt, hover over your profile image in the upper right corner and click on API dashboard in the appearing menu.

## Fetch posts of today

In [ ]:
api_token_ph = "PH_TOKEN"
api_url_posts = "https://api.producthunt.com/v1/posts"

# Construct header 
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + api_token_ph,
    'Host': 'api.producthunt.com'
}

# API call
ph_response = requests.get(api_url_posts, headers=headers)
ph_response.json()['posts'][0].keys()

## Create dict with all makers launching today

In [ ]:
maker_dict = {}
for product in ph_response.json()['posts']:
    for maker in product['makers']:

        if maker['twitter_username']: # Only save if maker has a Twitter handle
            
            product_data = {
                "prod_name": product['name'],
                "prod_tagline": product['tagline'], 
                "maker_name": maker['name'], 
                "twitter": maker['twitter_username']
            }
            
            if maker['twitter_username'] in maker_dict:
                maker_dict[maker['twitter_username']].append(product_data)
            else:
                maker_dict[maker['twitter_username']] = [product_data]

# Create a set with each Twitter handle
maker_twitter_handle_set = set(maker_dict.keys())

# 2. Fetch a list of all your Twitter follows
To match your Twitter followings with the makers launching on Product Hunt, we'll call an API endpoint that returns all your followings on Twitter.

## Get numerical username id for your Twitter user

In [ ]:
username = "YOUR_TWITTER_USERNAME"
twitter_bearer_token = "YOUR_BEARER_TOKEN"

search_url = f"https://api.twitter.com/2/users/by/username/{username}"
search_headers = {
    'Authorization': 'Bearer {}'.format(twitter_bearer_token)    
}
search_resp = requests.get(
    search_url,
    headers=search_headers, 
    params={
        "user.fields":"id" # Specifically ask for the id
    }
)
numerical_user_id = search_resp.json()['data']['id']

## Fetch your followings

In [ ]:
search_url = f"https://api.twitter.com/2/users/{numerical_user_id}/following"
search_headers = {
    'Authorization': 'Bearer {}'.format(twitter_bearer_token)    
}
search_resp = requests.get(
    search_url,
    headers=search_headers, 
    params={
        "user.fields":"username" # We only need the Twitter handle from the user fields
    }
)

## Create a set with Twitter handles for all users you're following

In [ ]:
twitter_following_set = set([user['username'] for user in search_resp.json()['data']])

# 3. Check if any maker is launching
Compare the sets. You'll get a new dict with Twitter handles if the sets match.

In [ ]:
matches = set(maker_twitter_handle_set) & set(twitter_following_set)
# eg {'twitter_handle_1','twitter_handle_2'}

# 4. Send an email if any maker is launching (optional)
If there is a match, let's compose an email.

In [ ]:
# Choose the AWS profile name for the boto3 setup
# Optional for sending email with SES
boto3.setup_default_session(profile_name='YOUR_AWS_ROLE_NAME')

In [ ]:
email_text = ""
for maker_id in matches:
    launch_text = f"The maker {maker_dict[maker_id][0]['maker_name']} @{maker_dict[maker_id][0]['twitter']} that you are following on Twitter just launched {len(maker_dict[maker_id]) + ' producs' if len(maker_dict[maker_id])>1 else 'a product'} on Product Hunt 🚀 "
    
    prod_texts = ""
    for product in maker_dict[maker_id]:
        prod_texts += f"Product mame: {product['prod_name']}"
        prod_texts += f"Product tagline: {product['prod_tagline']}"
    
    email_text += launch_text + prod_texts

## Send email with AWS SES

In [ ]:
ses_client = boto3.client('ses')

sender_email = "FROM_EMAIL_ADDRESS"
receiver_email = "TO_EMAIL_ADDRESS"
subject = "EMAIL_SUBJECT"

email_confirmation = ses_client.send_email(
    Destination={
        'BccAddresses': [],
        'CcAddresses': [],
        'ToAddresses': [receiver_email],
    },
    Message={
        'Body': {
            'Html': {
                'Charset': 'UTF-8',
                'Data': email_text,
            },
            'Text': {
                'Charset': 'UTF-8',
                'Data': email_text,
            },
        },
        'Subject': {
            'Charset': 'UTF-8',
            'Data': subject,
        },
    },
    ReplyToAddresses=[],
    Source=sender_email,
)